## Interim Model
 **[Target, Result, Analysis] at the end**



In [34]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from prettytable import PrettyTable

In [35]:
# Train Phase transformations
train_transforms = transforms.Compose([
                                      #  transforms.Resize((28, 28)),
                                      #  transforms.ColorJitter(brightness=0.10, contrast=0.1, saturation=0.10, hue=0.1),
                                       # transforms.RandomRotation((-7.0, 7.0), fill=(1,)),
                                       transforms.RandomRotation((-7.0, 7.0), fill=(1,)),
                                       transforms.ToTensor(),
                                       transforms.Normalize((0.1307,), (0.3081,)) # The mean and std have to be sequences (e.g., tuples), therefore you should add a comma after the values.
                                       # Note the difference between (0.1307) and (0.1307,)
                                       ])

# Test Phase transformations
test_transforms = transforms.Compose([
                                      #  transforms.Resize((28, 28)),
                                      #  transforms.ColorJitter(brightness=0.10, contrast=0.1, saturation=0.10, hue=0.1),
                                       transforms.ToTensor(),
                                       transforms.Normalize((0.1307,), (0.3081,))
                                       ])


In [36]:
train = datasets.MNIST('./data', train=True, download=True, transform=train_transforms)
test = datasets.MNIST('./data', train=False, download=True, transform=test_transforms)

In [37]:
SEED = 1

# CUDA?
device = torch.device("mps" if torch.backends.mps.is_available() else "cuda" if torch.cuda.is_available() else "cpu")
cuda = torch.cuda.is_available()
print("GPU Available?", device)

# For reproducibility
torch.manual_seed(SEED)

if device == "cuda":
    torch.cuda.manual_seed(SEED)
elif device == "mps":
    torch.manual_seed(SEED)

# dataloader arguments - something you'll fetch these from cmdprmt
dataloader_args = dict(shuffle=True, batch_size=128, num_workers=4, pin_memory=True) if device else dict(shuffle=True, batch_size=64)

# train dataloader
train_loader = torch.utils.data.DataLoader(train, **dataloader_args)

# test dataloader
test_loader = torch.utils.data.DataLoader(test, **dataloader_args)

# Pretty table for collecting all the accuracy and loss parameters in a table
log_table = PrettyTable()

GPU Available? mps


In [39]:
dropout_value = 0.01
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # Input Block
        self.convblock1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=8, kernel_size=(3, 3), padding=0, bias=False),
            nn.ReLU(),
            nn.BatchNorm2d(8),
            nn.Dropout(dropout_value)
        ) # output_size = 26

        # CONVOLUTION BLOCK 1
        self.convblock2 = nn.Sequential(
            nn.Conv2d(in_channels=8, out_channels=16, kernel_size=(3, 3), padding=0, bias=False),
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Dropout(dropout_value)
        ) # output_size = 24

         # TRANSITION BLOCK 1
        self.pool1 = nn.MaxPool2d(2, 2) # output_size = 12
        self.convblock3 = nn.Sequential(
            nn.Conv2d(in_channels=16, out_channels=8, kernel_size=(1, 1), padding=0, bias=False),
            nn.ReLU()
        ) # output_size = 12

        # CONVOLUTION BLOCK 2
        self.convblock4 = nn.Sequential(
            nn.Conv2d(in_channels=8, out_channels=12, kernel_size=(3, 3), padding=0, bias=False),
            nn.ReLU(),
            nn.BatchNorm2d(12),
            nn.Dropout(dropout_value)
        ) # output_size = 10

        self.convblock5 = nn.Sequential(
            nn.Conv2d(in_channels=12, out_channels=12, kernel_size=(3, 3), padding=0, bias=False),
            nn.ReLU(),
            nn.BatchNorm2d(12),
            nn.Dropout(dropout_value)
        ) # output_size = 8

        self.convblock6 = nn.Sequential(
            nn.Conv2d(in_channels=12, out_channels=16, kernel_size=(3, 3), padding=0, bias=False),
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Dropout(dropout_value)
        ) # output_size = 6

        # OUTPUT BLOCK
        self.convblock7 = nn.Sequential(
            nn.Conv2d(in_channels=16, out_channels=16, kernel_size=(3, 3), padding=1, bias=False),
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Dropout(dropout_value)
        ) # output_size = 6

        self.gap = nn.Sequential(
            nn.AvgPool2d(kernel_size=6) # 7>> 9... nn.AdaptiveAvgPool((1, 1))
        ) # output_size = 1

        self.convblock8 = nn.Sequential(
            
            nn.Conv2d(in_channels=16, out_channels=10, kernel_size=(1, 1), padding=0, bias=False),
            # nn.BatchNorm2d(10), NEVER
            # nn.ReLU() NEVER!
        ) # output_size = 1


    def forward(self, x):
        x = self.convblock1(x)
        x = self.convblock2(x)

        x = self.pool1(x)
        x = self.convblock3(x) #transition

        x = self.convblock4(x)
        x = self.convblock5(x)
        x = self.convblock6(x)
        x = self.convblock7(x) #padding=1

        x = self.gap(x)
        x = self.convblock8(x) #transition
        x = x.view(-1, 10)
        return F.log_softmax(x, dim=-1)

In [40]:
from torchsummary import summary
use_cuda = torch.cuda.is_available()
cuda = torch.device("cuda" if use_cuda else "cpu")
print(cuda)
model = Net().to(cuda)
summary(model, input_size=(1, 28, 28))

cpu
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 26, 26]              72
              ReLU-2            [-1, 8, 26, 26]               0
       BatchNorm2d-3            [-1, 8, 26, 26]              16
           Dropout-4            [-1, 8, 26, 26]               0
            Conv2d-5           [-1, 16, 24, 24]           1,152
              ReLU-6           [-1, 16, 24, 24]               0
       BatchNorm2d-7           [-1, 16, 24, 24]              32
           Dropout-8           [-1, 16, 24, 24]               0
         MaxPool2d-9           [-1, 16, 12, 12]               0
           Conv2d-10            [-1, 8, 12, 12]             128
             ReLU-11            [-1, 8, 12, 12]               0
           Conv2d-12           [-1, 12, 10, 10]             864
             ReLU-13           [-1, 12, 10, 10]               0
      BatchNorm2d-14           [-1,

In [41]:
from tqdm import tqdm

train_losses = []
test_losses = []
train_acc = []
test_acc = []

def train(model, device, train_loader, optimizer, epoch):
  model.train()
  pbar = tqdm(train_loader)
  correct = 0
  processed = 0
  for batch_idx, (data, target) in enumerate(pbar):
    # get samples
    data, target = data.to(device), target.to(device)

    # Init
    optimizer.zero_grad()
    # In PyTorch, we need to set the gradients to zero before starting to do backpropragation because PyTorch accumulates the gradients on subsequent backward passes.
    # Because of this, when you start your training loop, ideally you should zero out the gradients so that you do the parameter update correctly.

    # Predict
    y_pred = model(data)

    # Calculate loss
    loss = F.nll_loss(y_pred, target)
    train_losses.append(loss)

    # Backpropagation
    loss.backward()
    optimizer.step()

    # Update pbar-tqdm

    pred = y_pred.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
    correct += pred.eq(target.view_as(pred)).sum().item()
    processed += len(data)

    pbar.set_description(desc= f'Loss={loss.item()} Batch_id={batch_idx} Accuracy={100*correct/processed:0.2f}')
    train_acc.append(100*correct/processed)

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    test_losses.append(test_loss)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

    test_acc.append(100. * correct / len(test_loader.dataset))

In [42]:
print("model running on: ", device)
log_table = PrettyTable()
log_table.field_names = ["Epoch", "Training Accuracy", "Test Accuracy", "Diff", "Training Loss", "Test Loss"]

model =  Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

EPOCHS = 15
for epoch in range(EPOCHS):
    print("EPOCH:", epoch)
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)
    log_table.add_row([epoch, f"{train_acc[-1]:.2f}%", f"{test_acc[-1]:.2f}%", f"{float(train_acc[-1]) - float(test_acc[-1]):.2f}" ,f"{train_losses[-1]:.4f}", f"{test_losses[-1]:.4f}"])
print(log_table)

model running on:  mps
EPOCH: 0


Loss=0.12591548264026642 Batch_id=468 Accuracy=88.16: 100%|██████████| 469/469 [00:08<00:00, 53.68it/s] 



Test set: Average loss: 0.1030, Accuracy: 9717/10000 (97.17%)

EPOCH: 1


Loss=0.06678164005279541 Batch_id=468 Accuracy=97.83: 100%|██████████| 469/469 [00:08<00:00, 55.92it/s] 



Test set: Average loss: 0.0510, Accuracy: 9861/10000 (98.61%)

EPOCH: 2


...


Test set: Average loss: 0.0261, Accuracy: 9919/10000 (99.19%)

EPOCH: 13


Loss=0.007783832494169474 Batch_id=468 Accuracy=99.18: 100%|██████████| 469/469 [00:08<00:00, 52.86it/s] 



Test set: Average loss: 0.0217, Accuracy: 9928/10000 (99.28%)

EPOCH: 14


Loss=0.018775930628180504 Batch_id=468 Accuracy=99.24: 100%|██████████| 469/469 [00:08<00:00, 53.98it/s] 



Test set: Average loss: 0.0213, Accuracy: 9932/10000 (99.32%)

+-------+-------------------+---------------+-------+---------------+-----------+
| Epoch | Training Accuracy | Test Accuracy |  Diff | Training Loss | Test Loss |
+-------+-------------------+---------------+-------+---------------+-----------+
|   0   |       88.16%      |     97.17%    | -9.01 |     0.1259    |   0.1030  |
|   1   |       97.83%      |     98.61%    | -0.78 |     0.0668    |   0.0510  |
|   2   |       98.31%      |     98.51%    | -0.20 |     0.0186    |   0.0499  |
|   3   |       98.45%      |     98.98%    | -0.53 |     0.0682    |   0.0353  |
|   4   |       98.67%      |     98.97%    | -0.30 |     0.0135    |   0.0356  |
|   5   |       98.81%      |     99.15%    | -0.34 |     0.0340    |   0.0280  |
|   6   |       98.87%      |     99.08%    | -0.21 |     0.0114    |   0.0302  |
|   7   |       98.89%      |     98.90%    | -0.01 |     0.0280    |   0.0341  |
|   8   |       99.01%      |     

### TARGET
- Here we introduced Batch Normalisation in all the layers except Transition layer and last Output layer

### RESULT

- After adding **Batch Normalisation**

    - Parameters            : **7864**

    - Best Train Accuracy   : **99.51%**

    - Best Test Accuracy    : **99.31%**
---
- After adding **Dropout**

    - Parameters            : **7864**

    - Best Train Accuracy   : **99.42%**

    - Best Test Accuracy    : **99.26%**

---
    

- After adding **Image Augmentation**

    - Parameters            : **7864**

    - Best Train Accuracy   : **99.24%**

    - Best Test Accuracy    : **99.35%**

### Analysis
- **Model Architecture**
   
    - **Channels** :  1 → 8 → 16 → MaxPool → 16 → (Transition) → 8 → 12 → 12 → 16 → 16 → GAP → 16 → Transition → 10 

    - **Receptive Field**

        | Block               | Layer      | Input Size | Kernel x Stride | Padding | Input Channel | Output Channel | Output Size | Receptive Field |
        | ------------------- | ---------- | ---------- | --------------- | ------- | ------------- | -------------- | ----------- | --------------- |
        | Input Block         | Conv1      | 28         | 3 x 1           | 0       | 1             | 8              | 26          | 3               |
        | Convolution Block 1 | Conv2      | 26         | 3 x 1           | 0       | 8             | 16             | 24          | 5               |
        | Transition Block 1  | Pool1      | 24         | 2 x 2           | 0       | 16            | 16             | 12          | 6               |
        | Transition Block 1  | Conv3      | 12         | 1 x 1           | 0       | 16            | 8              | 12          | 6               |
        | Convolution Block 2 | Conv4      | 12         | 3 x 1           | 0       | 8             | 12             | 10          | 10              |
        | Convolution Block 2 | conv5      | 10         | 3 x 1           | 0       | 12            | 12             | 8           | 14              |
        | Convolution Block 2 | Conv6      | 8          | 3 x 1           | 0       | 12            | 16             | 6           | 18              |
        | Convolution Block 2 | Conv7      | 6          | 3 x 1           | 1       | 16            | 16             | 6           | 22              |
        | Output Block        | GAP        | 6          | 3 x 1           | 0       | 16            | 16             | 1           | 28              |
        | Output Block        | Conv8      | 1          | 1 x 1           | 0       | 16            | 10             | 1           | 28              |

- **Observation**
    - After introducing batch normalisation, the model is fitting very well. 

    - It has got 99.51% as training accuracy and 99.31% of Test accuracy. Also gap between both accuracy is very narrow.

    - This is fairly well but still not achived the required target of 99.4%.

    - Since there is little over-fitting, we will introduce some regularisation so that model can generaize the leraning and test accuracy can be improved.
    
    - We will add little dropout to all the layers.

    - **Output After Adding Batch Normalisation**

        | Epoch | Training Accuracy | Test Accuracy |  Diff | Training Loss | Test Loss |
        |-------|-------------------|---------------|-------|---------------|-----------|
        |   0   |       90.57%      |     97.68%    | -7.11 |     0.0925    |   0.0919  |
        |   1   |       98.02%      |     98.62%    | -0.60 |     0.0746    |   0.0579  |
        |   2   |       98.51%      |     98.46%    |  0.05 |     0.0242    |   0.0543  |
        |   3   |       98.73%      |     98.90%    | -0.17 |     0.0423    |   0.0377  |
        |   4   |       98.88%      |     99.09%    | -0.21 |     0.0822    |   0.0332  |
        |   5   |       99.01%      |     99.02%    | -0.01 |     0.0117    |   0.0344  |
        |   6   |       99.10%      |     98.94%    |  0.16 |     0.0707    |   0.0347  |
        |   7   |       99.20%      |     99.08%    |  0.12 |     0.0288    |   0.0302  |
        |   8   |       99.21%      |     98.86%    |  0.35 |     0.0418    |   0.0371  |
        |   9   |       99.28%      |     99.22%    |  0.06 |     0.0269    |   0.0278  |
        |   10  |       99.34%      |     99.15%    |  0.19 |     0.0099    |   0.0264  |
        |   11  |       99.38%      |     99.30%    |  0.08 |     0.0170    |   0.0239  |
        |   12  |       99.37%      |     99.12%    |  0.25 |     0.0027    |   0.0278  |
        |   13  |       99.39%      |     99.31%    |  0.08 |     0.0268    |   0.0255  |
        |   14  |       99.51%      |     99.27%    |  0.24 |     0.0292    |   0.0261  |

    ***

    

    - Even after adding **dropout** training is happening more robustly. Regularisation is going well. But the accuracy didn;t improve much. 

    - Lets add augmentation techniques to make the training more robust.


    - **Output after Adding Dropout**

        | Epoch | Training Accuracy | Test Accuracy |  Diff | Training Loss | Test Loss |
        |-------|-------------------|---------------|-------|---------------|-----------|
        |   0   |       88.25%      |     97.53%    | -9.28 |     0.0970    |   0.0918  |
        |   1   |       98.10%      |     98.55%    | -0.45 |     0.0282    |   0.0499  |
        |   2   |       98.61%      |     98.86%    | -0.25 |     0.0223    |   0.0432  |
        |   3   |       98.76%      |     98.95%    | -0.19 |     0.0269    |   0.0373  |
        |   4   |       98.86%      |     99.16%    | -0.30 |     0.0650    |   0.0289  |
        |   5   |       98.99%      |     98.96%    |  0.03 |     0.0113    |   0.0332  |
        |   6   |       99.08%      |     99.07%    |  0.01 |     0.0730    |   0.0330  |
        |   7   |       99.11%      |     99.18%    | -0.07 |     0.0026    |   0.0269  |
        |   8   |       99.18%      |     99.00%    |  0.18 |     0.0124    |   0.0313  |
        |   9   |       99.20%      |     99.13%    |  0.07 |     0.0496    |   0.0285  |
        |   10  |       99.26%      |     99.21%    |  0.05 |     0.0219    |   0.0259  |
        |   11  |       99.32%      |     99.21%    |  0.11 |     0.0503    |   0.0263  |
        |   12  |       99.34%      |     99.07%    |  0.27 |     0.0027    |   0.0302  |
        |   13  |       99.42%      |     99.19%    |  0.23 |     0.0011    |   0.0239  |
        |   14  |       99.37%      |     99.26%    |  0.11 |     0.0339    |   0.0279  |


    ***

    - After adding **image augmentaion** in training dataset, Test accuracy increased a little more. 

    - It achived test accuracy of 99.35%.

    - But one observation is the accuracy is fluctuating between values. Hence step scheduler can help.


        | Epoch | Training Accuracy | Test Accuracy |  Diff | Training Loss | Test Loss |
        |-------|-------------------|---------------|-------|---------------|-----------|
        |   0   |       88.16%      |     97.17%    | -9.01 |     0.1259    |   0.1030  |
        |   1   |       97.83%      |     98.61%    | -0.78 |     0.0668    |   0.0510  |
        |   2   |       98.31%      |     98.51%    | -0.20 |     0.0186    |   0.0499  |
        |   3   |       98.45%      |     98.98%    | -0.53 |     0.0682    |   0.0353  |
        |   4   |       98.67%      |     98.97%    | -0.30 |     0.0135    |   0.0356  |
        |   5   |       98.81%      |     99.15%    | -0.34 |     0.0340    |   0.0280  |
        |   6   |       98.87%      |     99.08%    | -0.21 |     0.0114    |   0.0302  |
        |   7   |       98.89%      |     98.90%    | -0.01 |     0.0280    |   0.0341  |
        |   8   |       99.01%      |     99.18%    | -0.17 |     0.0884    |   0.0251  |
        |   9   |       99.07%      |     99.25%    | -0.18 |     0.1087    |   0.0241  |
        |   10  |       99.07%      |     99.21%    | -0.14 |     0.0366    |   0.0246  |
        |   11  |       99.12%      |     99.35%    | -0.22 |     0.0081    |   0.0212  |
        |   12  |       99.19%      |     99.19%    |  0.00 |     0.0070    |   0.0261  |
        |   13  |       99.18%      |     99.28%    | -0.10 |     0.0078    |   0.0217  |
        |   14  |       99.24%      |     99.32%    | -0.08 |     0.0188    |   0.0213  |
